# Import data

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
values = pd.read_csv('csv_files/training_set_values.csv')
labels = pd.read_csv('csv_files/training_set_labels.csv')

In [3]:
# Merge my predicors and target variable
df = pd.merge(values, labels, on = 'id')

## View data frame and columns

In [4]:
df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [5]:
for col in df.columns:
    print('-------------------')
    print(col, '\n')
    print(df[col].value_counts(normalize=True).head())
    print('--------------------')

-------------------
id 

2047     0.000017
72310    0.000017
49805    0.000017
51852    0.000017
62091    0.000017
Name: id, dtype: float64
--------------------
-------------------
amount_tsh 

0.0       0.700993
500.0     0.052222
50.0      0.041616
1000.0    0.025051
20.0      0.024630
Name: amount_tsh, dtype: float64
--------------------
-------------------
date_recorded 

2011-03-15    0.009630
2011-03-17    0.009394
2013-02-03    0.009192
2011-03-14    0.008754
2011-03-16    0.008636
Name: date_recorded, dtype: float64
--------------------
-------------------
funder 

Government Of Tanzania    0.162898
Danida                    0.055841
Hesawa                    0.039487
Rwssp                     0.024639
World Bank                0.024191
Name: funder, dtype: float64
--------------------
-------------------
gps_height 

 0     0.344074
-15    0.001010
-16    0.000926
-13    0.000926
-20    0.000875
Name: gps_height, dtype: float64
--------------------
-------------------
installe

In [6]:
# Look for missing data
df.isna().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

# Seperate date into year and month columns

In [7]:
df.drop(['date_recorded', 'id', ],axis=1, inplace=True)

# Fill missing data points

In [8]:
df['funder'].fillna('Government Of Tanzania', inplace=True)

In [9]:
df.installer.fillna('DWE', inplace=True)

In [10]:
df.dropna(subset=['subvillage'], inplace=True)

In [11]:

# fill missing public_meeting values with either True or False based on probability
df['public_meeting'].fillna('?', inplace=True)

pm_prob = [0.909838, 0.090162]
prob = [i/sum(pm_prob) for i in pm_prob]

def impute_value(value):
    if value == '?':
        return np.random.choice(['True', 'False'], p = prob)
    else:
        return value
    
df.public_meeting = df.public_meeting.map(lambda x: impute_value(x))

In [12]:
df.scheme_management.fillna('VWC', inplace=True)

In [13]:
df.drop('scheme_name', axis = 1,  inplace=True)

In [14]:
# fill missing permis values with either True or False based on probability
df['permit'].fillna('?', inplace=True)

permit_prob = [ 0.68955, 0.31045]
prob = [i/sum(permit_prob) for i in permit_prob]

    
df.permit = df.permit.map(lambda x: impute_value(x))

In [15]:
df.to_csv('csv_files/training_data_clean')